Definitions:

In [24]:
# Define the State class representing an individual state in an NFA
class State:
    state_counter = 0  # Static counter to assign unique IDs to states
    
    def __init__(self):
        self.transitions = {}  # Dictionary to store transitions from this state
        self.id = State.state_counter  # Unique ID for this state
        self.is_final = False  # Flag to indicate if this state is a final state
        State.state_counter += 1  # Increment the state counter


# Define the NFA class representing a non-deterministic finite automaton
class NFA:
    def __init__(self):
        self.states = []  # List to store states of the NFA
        self.alphabet = set()  # Set to store the alphabet of the NFA
        self.start_state = None  # The start state of the NFA
    
    # Method to print the NFA details
    def print(self):
        print("Alphabet:", ' '.join(sorted(self.alphabet)))  # Print the alphabet
        print("States:", ' '.join([f'q{i}' for i in range(len(self.states))]))  # Print the states
        print("Start state: q0")  # Print the start state
        print("Final states:", ' '.join([f'q{i}' for i, state in enumerate(self.states) if state.is_final]))  # Print the final states
        print("Transitions:")
        # Print each transition
        for i, state in enumerate(self.states):
            for symbol, next_states in state.transitions.items():
                for next_state in next_states:
                    print(f"q{i} {symbol} q{self.states.index(next_state)}")


Parse Regex: 

In [25]:
# Function to convert a regex to an NFA
def regex_to_nfa(regex: str):
    nfa = NFA()  # Create a new NFA
    nfa.start_state, final_state, idx = parse_regex(regex, nfa, 0)  # Parse the regex to populate the NFA
    final_state.is_final = True  # Mark the final state as final
    return nfa  # Return the NFA


# Function to parse the regex and construct the NFA
def parse_regex(regex: str, nfa: NFA, start_idx: int):
    start = State()  # Create a start state
    nfa.states.append(start)  # Add the start state to the NFA
    current = start  # Set the current state to the start state
    past = None  # Variable to keep track of the previous state
    
    i = start_idx  # Initialize the index
    while i < len(regex):
        if regex[i].isalpha(): 
            nfa.alphabet.add(regex[i])  # Add the symbol to the alphabet
            next_state = State()  # Create the next state
            nfa.states.append(next_state)  # Add the next state to the NFA
            current.transitions.setdefault(regex[i], []).append(next_state)  # Add the transition
            past = current  # Update the past state
            current = next_state  # Update the current state
            
        elif regex[i] == '*' and past is not None:
            current.transitions.setdefault('ε', []).append(past)  # Add epsilon transition from current to past
            past.transitions.setdefault('ε', []).append(current)  # Add epsilon transition from past to current
        
        elif regex[i] == '|':
            next_start, next_end, j = parse_regex(regex, nfa, i + 1)  # Parse the sub-expression
            start.transitions.setdefault('ε', []).append(next_start)  # Add epsilon transition from start to sub-start
            next_state = State()  # Create the next state
            nfa.states.append(next_state)  # Add the next state to the NFA
            current.transitions.setdefault('ε', []).append(next_state)  # Add epsilon transition from current to next
            next_end.transitions.setdefault('ε', []).append(next_state)  # Add epsilon transition from sub-end to next
            return start, next_state, j  # Return the start, end states, and index
        
        elif regex[i] == '+' and past is not None:
            current.transitions.setdefault('ε', []).append(past)  # Add epsilon transition from current to past
        
        elif regex[i] == '(':
            sub_start, sub_end, j = parse_regex(regex, nfa, i + 1)  # Parse the sub-expression
            current.transitions.setdefault('ε', []).append(sub_start)  # Add epsilon transition from current to sub-start
            past = current  # Update the past state
            current = sub_end  # Update the current state
            i = j  # Update the index
            
        elif regex[i] == ')':
            return start, current, i  # Return the start, end states, and index
        
        i += 1  # Move to the next character
        
    return start, current, i  # Return the start, end states, and index

Validator: 

In [26]:
# Function to validate the input regex
import re

def is_valid_regex(input_string: str) -> bool:
    # Define the regex pattern for valid characters
    pattern = r"^[a-z+\*\(\)]*$"
    
    # Check if the input string contains only valid characters
    if not re.match(pattern, input_string):
        return False

    # Function to check for balanced parentheses
    def are_parentheses_balanced(s):
        stack = []
        for char in s:
            if char == '(':
                stack.append(char)
            elif char == ')':
                if not stack:
                    return False
                stack.pop()
        return not stack

    # Check if the parentheses are balanced
    if not are_parentheses_balanced(input_string):
        return False

    return True

Input Section:

In [27]:
# Input section to get the regex from the user
regex = input()
regex = regex.replace(" ", "")  # Remove any whitespace from the input
if not is_valid_regex(regex):
    print("Invalid")
    raise ValueError("Invalid regex provided")  # Raise an error if the regex is invalid

nfa = regex_to_nfa(regex)  # Convert the regex to an NFA
nfa.print()  # Print the NFA details

Alphabet: a b
States: q0 q1 q2
Start state: q0
Final states: q2
Transitions:
q0 a q1
q1 ε q0
q1 b q2
